In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks such as summarization is hard enoough.

In [ ]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [ ]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  #Your code here
  v = CountVectorizer()
  text = [sub['text'] for sub in corpus]
  texts = v.fit_transform(text)
  genres = [sub['genre'] for sub in corpus]
  vocab = v.get_feature_names()
  return texts,genres,vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createInterestingFeatures(corpus):
  #Your coode here

  #Vectorizing
  v1 = CountVectorizer(stop_words= stpwrds) 
  text = [sub['text'] for sub in corpus]
  texts = v1.fit_transform(text)
  genres = [sub['genre'] for sub in corpus]
  vocab = v1.get_feature_names()

  #removing stopwords
  for i in range(0,len(corpus)):
    lst = word_tokenize(corpus[i]['text'])
    res = [word for word in lst if word.lower() not in stpwrds]
    corpus[i]['text']= " ".join(res)
  
  #lemmatizing
  lemmatizer = WordNetLemmatizer()
  lmtz = [' '.join([lemmatizer.lemmatize(word, pos='a') for word in corpus[i]['text'].split()]) for i in range(0,len(corpus))]
  for i in range(0,len(corpus)):
    corpus[i]['text'] = lmtz[i]

  return texts, genres, vocab


In [ ]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  

In [ ]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [ ]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.807692
The most informative terms for pos are: ['helena', 'prospero', 'sir', 'you', 'for', 'your', 'duke', 'me', 'of', 'love', 'preserver', 'preserved', 'preserve', 'preserv', 'preservation', 'preservers', 'presents', 'presentment', 'presently', 'presenting']
The most informative terms for neg are: ['our', 'him', 'rom', 'iago', 'thy', 'ham', 'what', 'imogen', 'his', 'brutus', 'lear', 'timon', 'preservers', 'preserver', 'preserved', 'preserv', 'preservative', 'preservation', 'presents', 'presentment']
----------L2 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: ['you', 'prospero', 'duke', 'helena', 'antonio', 'me', 'for', 'your', 'sir', 'ariel', 'sebastian', 'hermia', 'lysander', 'parolles', 'stephano', 'will', 'leontes', 'caliban', 'demetrius', 'love']
The most informative terms for neg are: ['ham', 'iago', 'him', 'our', 'othello', 'what', 'his', 'lear', 'imogen', 'brutus', '

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw =list(stopwords.words("english"))
stpwrds = sw + ["art","doth", "dost", "ere", "hast", "hath", "hence", "hither", "nigh", "oft", "shouldst", "thither", "tither", "thee", "thou",
                "thine", "thy", "tis", "twas", "wast", "whence", "wherefore", "whereto", "withal", "wouldst", "ye", "yon", "yonder","him","our","his","to","your","you","for","and"]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [ ]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.461538
The most informative terms for pos are: ['antonio', 'sir', 'love', 'helena', 'leontes', 'prospero', 'duke', 'pedro', 'syracuse', 'master', 'petruchio', 'lysander', 'hermia', 'presenting', 'presenteth', 'presenters', 'presently', 'present', 'presentation', 'presentment']
The most informative terms for neg are: ['rome', 'lear', 'imogen', 'troilus', 'rom', 'iago', 'macbeth', 'nurse', 'ham', 'caesar', 'timon', 'th', 'romeo', 'othello', 'lord', 'presenteth', 'presenting', 'presenters', 'present', 'presentation']
----------L2 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: ['sir', 'duke', 'helena', 'antonio', 'prospero', 'leontes', 'love', 'parolles', 'camillo', 'hermia', 'lysander', 'master', 'rosalind', 'bertram', 'ariel', 'sebastian', 'clown', 'portia', 'stephano', 'pedro']
The most informative terms for neg are: ['macbeth', 'ham', 'timon', 'lear', 'rom', 'nurse', 'romeo'

Although the accuracy has gone down for createInterestingFeatures(), the most informative terms for both positive and negative classes have improved after removing stopwords and lemmatizing. For example, the most informative terms for negative classes are macbeth, timon, lear, romeo etc, who are all characters in tragic plays, hence classified as negative terms